# Answer Coalescence

Strider finds and ranks all paths that fulfill the query.  Some of these answers can be very similar.  For instance, two answers may differ only by a gene node, but perhaps those genes are from the same family.  This permits a higher-level, more parsimonious description of an answer set.

In [1]:
import requests
import json
from notebook_functions import reasonerGraphToCytoscape, knowledgeGraphToCytoscape, answerGraphToCytoscape
from cyjupyter import Cytoscape
import time

In [2]:
def getlabel(curie):
    url = 'https://nodenormalization-sri.renci.org/get_normalized_nodes'
    r = requests.get(url,params={'curie':curie}).json()
    try:
        return r[curie]['id']['label']
    except:
        return ''

class Strider:
    def __init__(self):
        self.url='http://robokop.renci.org:5781'
    def call(self,question):
        message = {'message': {'query_graph': question}}
        return self.send_message(message)
    def send_message(self,message):
        response = requests.post(f'{self.url}/query',json=message)
        if response.status_code == 200:
            pid = response.json()
            return pid
        else:
            print(response.status_code)
            return None
    def query_result(self,pid):
        r = requests.get(f'{self.url}/results',params={'query_id':pid})
        return r.json()
    
strider = Strider()

In [3]:
def create_question(relation_type,disease_curie=None,chem_curie=None):
    question = { "nodes": [{"id":"n0", "type":"disease"},
                          {"id": "n1", "type":"chemical_substance"}],
                 "edges": [ {"id": "e0", "source_id": "n1", "target_id": "n0", "type": relation_type}]}
    if disease_curie is not None:
        question['nodes'][0]['curie']=disease_curie
    if chem_curie is not None:
        question['nodes'][1]['curie']=chem_curie
    return question

In [4]:
disease = "MONDO:0005148"   #T2D
edgetype="treats"
question = create_question(edgetype,disease_curie=disease)
message = {"message": {"query_graph": question}}

c=reasonerGraphToCytoscape(question)
Cytoscape(data=c, visual_style=c["style"], layout_name='grid', layout={"name": "grid", "height": "200px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'n0', 'label': 'disease\n[MONDO:0005148]', 'curie': 'MO…

In [5]:
p = strider.send_message(message)
print(p)

79e4f0c9-1003-41a1-b474-8449be06e391


In [6]:
strider_result=strider.query_result(p)
while len(strider_result['results']) == 0:
    time.sleep(5)
    print('checking')
    strider_result = strider.query_result(p)
print('results found')

checking
results found


In [7]:
time.sleep(5)
strider_result = strider.query_result(p)
print(f"Found {len(strider_result['results'])} results")

Found 78 results


In [8]:
r0 = strider_result['results'][9]
c = answerGraphToCytoscape(r0,strider_result['knowledge_graph'])
Cytoscape(data=c, visual_style=c["style"], layout_name='grid', layout={"height": "100px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'MONDO:0005148', 'label': 'type 2 diabetes mellitus\n[M…

In [9]:
r0 = strider_result['results'][13]
c = answerGraphToCytoscape(r0,strider_result['knowledge_graph'])
Cytoscape(data=c, visual_style=c["style"], layout_name='grid', layout={"height": "100px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'MONDO:0005148', 'label': 'type 2 diabetes mellitus\n[M…

In [10]:
r0 = strider_result['results'][20]
c = answerGraphToCytoscape(r0,strider_result['knowledge_graph'])
Cytoscape(data=c, visual_style=c["style"], layout_name='grid', layout={"height": "100px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'MONDO:0005148', 'label': 'type 2 diabetes mellitus\n[M…

## Coalesce Answers by Property

In [11]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'property'}
p_r = requests.post(url,json=strider_result,params=params)
print(p_r.status_code)
property_result = p_r.json()

200


In [12]:
results = property_result['results']
for result in results:
    chemnode = result['node_bindings'][1]
    print(len(chemnode['kg_id']), chemnode['p_values'], chemnode['properties'])

23 [1.4507062494232832e-32] ['drugbank.investigational']
22 [2.9927753494707155e-30] ['drugbank.approved']
20 [1.2767864105470421e-24] ['therapeutic_flag']
10 [4.823367861758246e-24] ['hypoglycemic_agent']
8 [3.3494981019524477e-22, 8.916362091726007e-21] ['Drugs Used in Diabetes', 'Blood Glucose Lowering Agents']
22 [4.830130548044178e-22, 6.478713505765552e-22] ['drug', 'pharmaceutical']
24 [6.205859186704515e-21] ['application']
7 [8.254275649747387e-20] ['Oral Hypoglycemics']
14 [1.0939886086070841e-19] ['oral']
10 [1.3742711531874345e-19] ['Alimentary Tract and Metabolism']
14 [3.4597402886307423e-16] ['inhibitor']
9 [5.4258882586528116e-15] ['Enzyme Inhibitors']
9 [6.668684135065894e-15] ['Cytochrome P-450 Enzyme Inhibitors']
4 [5.936897949610144e-14, 5.594130321207606e-13, 2.7511305706926877e-10, 2.6960183242478735e-08] ['Sulfonylurea Compounds', 'Sulfonylureas', 'Urea', 'Sulfones']
12 [1.0815168960134938e-13] ['enzyme_inhibitor']
29 [1.3741445773945114e-13] ['biological_role']


In [13]:
r21 = results[21]
print(r21['node_bindings'][1]['properties'])
c = answerGraphToCytoscape(r21,property_result['knowledge_graph'])
Cytoscape(data=c, visual_style=c["style"], layout_name='cose', layout={"height": "500px"})

['Benzopyrans', 'Pyrans']


Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'MONDO:0005148', 'label': 'type 2 diabetes mellitus\n[M…

## Coalesce Answers by Ontology

In [14]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'ontology'}
o_r = requests.post(url,json=strider_result,params=params)
print(o_r.status_code)
ontology_result = o_r.json()
print( len(ontology_result['results']))

200
1


In [15]:
results = ontology_result['results']
for result in results:
    print('================')
    chemnode = result['node_bindings'][1]
    sclass = chemnode['superclass']
    label = getlabel(sclass)
    nodes = chemnode['kg_id']
    labels = {x: f'{getlabel(x)}' for x in nodes }
    for x in nodes:
        print(f'{labels[x]} ({x})')
    print('----have superclass----')
    print(f'{label} ({sclass})')

tolbutamide (CHEBI:27999)
chlorpropamide (CHEBI:3650)
glyburide (CHEBI:5441)
glipizide (CHEBI:5384)
----have superclass----
N-sulfonylurea (CHEBI:76983)


In [16]:
r0 = results[0]
c = answerGraphToCytoscape(r0,ontology_result['knowledge_graph'])
Cytoscape(data=c, visual_style=c["style"], layout_name='cose', layout={"height": "500px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'MONDO:0005148', 'label': 'type 2 diabetes mellitus\n[M…

## Coalescence by Graph

Coalescence by graph is similar to coalescence by ontology, but the new relationships can be more general than `is_a`. It is currently implemented as a third option to the AnswerCoalesence service.

In [17]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'graph'}
g_r = requests.post(url,json=strider_result,params=params)
print(g_r.status_code)
graph_result = g_r.json()
print( len(graph_result['results']))

200
79


In [22]:
results = graph_result['results']

cutoff = 1e-6
    
print(f'Linked node P-value cutoff is: {cutoff}')

for result in results:
    chemnode = result['node_bindings'][1]
    
    if chemnode['p_value'] > 0 and chemnode['p_value'] < cutoff:
        print('================')

        print(f"Linked node P-value: {chemnode['p_value']}")

        nodes = chemnode['kg_id']
        labels = {x: f'Node: {getlabel(x)}' for x in nodes }
           
        for x in nodes:
            print(f'{labels[x]} ({x})')
             
        enriched_nodes = chemnode['enriched_nodes']
        #enriched_labels = {x: f'{getlabel(x)}' for x in enriched_nodes }
        
        for x in enriched_nodes:
            print(f'Enriched node: {x}')

Linked node P-value cutoff is: 1e-06
Linked node P-value: 8.254846082252637e-171
Node: 2-dodecyl-6-methoxycyclohexa-2,5-diene-1,4-dione (MESH:C581182)
Node: orlistat (CHEBI:94686)
Node: curcumin (CHEBI:3962)
Node: lithocholic acid (CHEBI:16325)
Node: Drugs, Chinese Herbal (MESH:D004365)
Node: EC 3.4.15.1 (peptidyl-dipeptidase A) inhibitor (CHEBI:35457)
Node: hemin (CHEBI:50385)
Node: (-)-epigallocatechin 3-gallate (CHEBI:4806)
Node: Plant Extracts (MESH:D010936)
Node: nymphayol (MESH:C000594859)
Node: Fibric Acids (MESH:D058607)
Node: Lead (PUBCHEM:23927)
Node: metformin (CHEBI:6801)
Node: (S)-fenfluramine (CHEBI:439329)
Node: diosmin (CHEBI:4631)
Node: stannous chloride (MESH:C023599)
Node: (E)-cinnamaldehyde (CHEBI:16731)
Node: berberine (CHEBI:16118)
Node: fumosorinone (MESH:C000599091)
Node: chlorpropamide (CHEBI:3650)
Node: astaxanthin (CHEBI:40968)
Node: dibromine (CHEBI:29224)
Node: quinolines (CHEBI:26513)
Node: Panax notoginseng extract (MESH:C519374)
Node: trans-resveratrol (